# this notebook will demonstrate how to clean the Tencent Raw Data

the raw dataset in this project is big and messy. We will load it one by one and explain the features.

## load the libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from scipy.stats import norm
from scipy import stats
from loguru import logger

In [104]:
# open ad_static_feature file
collist = ['Id','Date','AdAccountId','CommodityId','CommodityType','AdIndustryId','AdSize']
with open('ad_static_feature.out') as f:
    df = pd.read_csv(f,sep='\t',names = collist)
# open ad_operation file
collist2 = ['Id','Date','Type','UpdateAttr','UpdateAttrValue']
with open('ad_operation.dat') as f:
    df2 = pd.read_csv(f,sep='\t',names = collist2)

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## understand the data

what we need in test dataset:
ID, ADID: discard before use
Date: date the ad is created
AdSize:
AdIndustryId:
CommodityType:
CommodityId:
AdAccountId:
ExposureTime:
Crowd:
Bid:

In [105]:
df.head()

,Id,Date,AdAccountId,CommodityId,CommodityType,AdIndustryId,AdSize
0,106452,1529958950,22226,16088,13,225,NaN
1,233649,1538221936,25681,7356,13,136,1
2,547531,1550731020,20696,-1,1,186,40
3,707841,1551857857,3968,-1,3,186,40
4,457009,1550439402,23614,7447,13,172,NaN


In [4]:
df.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735911 entries, 0 to 735910
Data columns (total 7 columns):
Id               735911 non-null int64
Date             735911 non-null int64
AdAccountId      735911 non-null int64
CommodityId      735911 non-null object
CommodityType    735911 non-null int64
AdIndustryId     735911 non-null object
AdSize           509252 non-null object
dtypes: int64(4), object(3)
memory usage: 119.8 MB


In [106]:
def create_ad_static_feature(df):
    # if AdSize = NaN, discard
    df = df.dropna(how='any')
    return df
    
df_ad_static_feature = create_ad_static_feature(df)

In [107]:
df_ad_static_feature.head()

,Id,Date,AdAccountId,CommodityId,CommodityType,AdIndustryId,AdSize
1,233649,1538221936,25681,7356,13,136,1
2,547531,1550731020,20696,-1,1,186,40
3,707841,1551857857,3968,-1,3,186,40
5,733436,1552977426,22405,31722,5,117,64
6,249105,1552641796,11360,29999,18,145,44


In [112]:
def create_ad_create(df2):
    ad_create = df2[df2['Type'] == 2]
    temp = ad_create.set_index(['Id','Date','Type','UpdateAttr']).unstack()
    temp.columns = [col for col in temp.columns.values]
    temp.columns = ['Bid','Crowd','Time']
    ad_create_sort = temp.reset_index()
    return ad_create_sort

df_ad_create = create_ad_create(df2)
# note when date = 0, we have duplicated results for the same id, 
# when the ad is create, date = 0, the initial setings.
# first check when ads are created
# for each new ad, three attributes are updated: 2,3,4. corresponding to: bid(unit cent),crowd,and exposuretime

In [113]:
df_ad_create.head()

,Id,Date,Type,Bid,Crowd,Time
0,31,0,2,100,area:11442,"281474976694272,281474976694272,28147497669427..."
1,32,0,2,83,area:7572,"70368475742208,70368475742208,70368475742208,7..."
2,69,0,2,180,area:7572,"281474976694272,281474976694272,28147497669427..."
3,84,0,2,100,area:6410,"4398045462528,4398045462528,4398045462528,4398..."
4,85,0,2,70,"age:217,601,202,837,942,638,287,5,394,347,731,...","281474976694272,281474976694272,28147497669427..."


In [158]:
# next we need to find the union of ad_create and ad_static_features

# this is when the ad is created, the features associated with it.

In [234]:
def create_ad_modify(df2):
    # get all modify histoy
    ad_modify = df2[df2['Type'] == 1]
    # change of the state value is not necessary information. discard
    ad_modify = ad_modify[ad_modify['UpdateAttr'] != 1]
    # convert date into epoch seconds
    temp = ad_modify['Date']
    ad_modify['Date'] = (pd.to_datetime(temp,format="%Y%m%d%H%M%S").astype(int) / 10**9).astype(int).astype(str)
    return ad_modify

df_ad_modify = create_ad_modify(df2)

In [235]:
def create_ad_merge(df_ad_static_feature,df_ad_create,df_ad_modify):
    merge1 = pd.merge(df_ad_static_feature,df_ad_create,how='inner',on='Id',sort=True)
    merge2 = pd.merge(merge1,df_ad_modify,how = 'left', on = 'Id')
    merge2_p1 = merge2[merge2['UpdateAttr'] == 2]
    merge2_p1['Bid'] = merge2_p1['UpdateAttrValue']
    merge2_p2 = merge2[merge2['UpdateAttr'] == 3]
    merge2_p2['Crowd'] = merge2_p2['UpdateAttrValue']
    merge2_p3 = merge2[merge2['UpdateAttr'] == 4]
    merge2_p3['Time'] = merge2_p3['UpdateAttrValue']
    merge2_p4 = merge2[merge2['UpdateAttr'].isnull()]
    merge_final = pd.concat([merge2_p1,merge2_p2,merge2_p3,merge2_p4])
    merge_final = merge_final.sort_values("Id",axis = 0)
    drop_list = ['Type_y','UpdateAttr','UpdateAttrValue','Date_y','Type_x']
    merge_final.drop(columns = drop_list, inplace = True)
    renamelist = ['Id','Date_x','Date','Bid','Crowd','Time','AdAccountId','CommodityId',
              'CommodityType','AdIndustryId','AdSize']
    merge_final = merge_final.reindex(columns = renamelist)
    merge_final = merge_final.sort_values(['Id','Date'],axis=0)
    return merge_final

df_ad_merge = create_ad_merge(df_ad_static_feature,df_ad_create,df_ad_modify)

    
    

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [237]:
df_ad_merge.head()

,Id,Date_x,Date,Bid,Crowd,Time,AdAccountId,CommodityId,CommodityType,AdIndustryId,AdSize
0,31,1552432970,NaN,100,area:11442,"281474976694272,281474976694272,28147497669427...",12577,18683,13,224,40
1,32,1543563617,1550727091,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752,32534,13,136,40
2,32,1543563617,1551110479,90,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752,32534,13,136,40
3,32,1543563617,1551245501,96,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752,32534,13,136,40
4,32,1543563617,1551331895,106,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752,32534,13,136,40


In [328]:
# next deal with totalexposure.log
# this file is huge...break it into chunks
row_count = 500000
expo_colname = ['AdRequestId','Time','LocationId','UserId','Id','AdSize','Bid','pctr','quality_ecpm','total_ecpm']
AdIdSet = set(df_ad_merge['Id'])
chunkcount = 0
for chunk in pd.read_csv('totalExposureLog.out', names = expo_colname, sep='\t', chunksize=row_count):
    x = chunk[chunk['Id'].isin(AdIdSet)]
    chunkcount += len(x)
    print("useful request in this chunk is ",chunkcount)
    x.to_hdf('store_test.h5', key = 'df',format='table',append=True, mode='a')

useful request in this chunk is  19620
useful request in this chunk is  39010
useful request in this chunk is  58199
useful request in this chunk is  77424
useful request in this chunk is  96984
useful request in this chunk is  116377
useful request in this chunk is  135586
useful request in this chunk is  159775
useful request in this chunk is  185847
useful request in this chunk is  212156
useful request in this chunk is  238431
useful request in this chunk is  264960
useful request in this chunk is  291053
useful request in this chunk is  317015
useful request in this chunk is  343499
useful request in this chunk is  370493
useful request in this chunk is  397471
useful request in this chunk is  424010
useful request in this chunk is  450909
useful request in this chunk is  477853
useful request in this chunk is  504538
useful request in this chunk is  531876
useful request in this chunk is  561727
useful request in this chunk is  591346
useful request in this chunk is  621064
usefu

useful request in this chunk is  5912016
useful request in this chunk is  5945800
useful request in this chunk is  5979661
useful request in this chunk is  6005879


In [326]:
opt = pd.read_hdf('store_test.h5', 'df')


In [327]:
opt.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504538 entries, 2 to 10499989
Data columns (total 10 columns):
AdRequestId     504538 non-null int64
Time            504538 non-null int64
LocationId      504538 non-null int64
UserId          504538 non-null int64
Id              504538 non-null int64
AdSize          504538 non-null int64
Bid             504538 non-null int64
pctr            504538 non-null float64
quality_ecpm    504538 non-null float64
total_ecpm      504538 non-null float64
dtypes: float64(3), int64(7)
memory usage: 42.3 MB


In [318]:
opt.shape

(100, 10)

In [322]:
x[200:300].to_hdf('test2.h5',key='df',append=True, mode='a', format='t')

In [323]:
opt2 = pd.read_hdf('test2.h5', 'df')

In [324]:
opt2.shape

(100, 10)